<a href="https://colab.research.google.com/github/MyatKaung/freecodecampMachineLearning/blob/main/fcc_predict_health_costs_with_regression_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Check the first 5 rows of the dataset
print(dataset.head())

# Check for missing values
print(dataset.isnull().sum())

# Check data types
print(dataset.dtypes)


In [ ]:
# Convert 'sex' column to numerical values
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})

# Convert 'smoker' column to numerical values
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})

# Convert 'region' column to numerical values using one-hot encoding
dataset = pd.get_dummies(dataset, columns=['region'])


In [ ]:
dataset.head()

In [ ]:
print(dataset.dtypes)

In [ ]:
# 1. Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# 2. Normalize the features
def norm(x, train_stats):
    return (x - train_stats['mean']) / train_stats['std']

train_stats = train_dataset.describe().transpose()

normed_train_data = norm(train_dataset, train_stats)
normed_test_data = norm(test_dataset, train_stats)

In [ ]:
normed_train_data.head()

In [ ]:
print(normed_train_data.dtypes)

In [ ]:
print(train_labels)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:


model = Sequential([
    Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(loss='mse',
              optimizer=tf.keras.optimizers.RMSprop(0.001),
              metrics=['mae', 'mse'])

history = model.fit(
    normed_train_data, train_labels,
    epochs=1000, validation_split=0.2, verbose=0)

# 4. Evaluate the model and plot the predictions
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)